In [1]:
import csv
import random
import math
def loadCsv(filename):
    filename="Ex05.csv"
    lines = csv.reader(open(filename, "rt"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated
def mean(numbers):
   return sum(numbers)/float(len(numbers))
def stdev(numbers):
   avg = mean(numbers)
   variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
   return math.sqrt(variance)
def summarize(dataset):
   summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
   del summaries[-1]
   return summaries
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    print(len(separated))
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)     
    print(summaries)
    return summaries
def calculateProbability(x, mean, stdev):
   exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
   return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent
def calculateClassProbabilities(summaries, inputVector):
   probabilities = {}
   for classValue, classSummaries in summaries.items():
      probabilities[classValue] = 1
      for i in range(len(classSummaries)):
         mean, stdev = classSummaries[i]
         x = inputVector[i]
         probabilities[classValue] *= calculateProbability(x, mean, stdev)
   return probabilities      
def predict(summaries, inputVector):
   probabilities = calculateClassProbabilities(summaries, inputVector)
   bestLabel, bestProb = None, -1
   for classValue, probability in probabilities.items():
      if bestLabel is None or probability > bestProb:
         bestProb = probability
         bestLabel = classValue
   return bestLabel
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
        print(i+1,': ', testSet[i],"--",result)
    return predictions
def getAccuracy(testSet, predictions):
   correct = 0
   for i in range(len(testSet)):
      if testSet[i][-1] == predictions[i]:
         correct += 1
   return (correct/float(len(testSet))) * 100.0
filename = 'diabetes1.csv'
splitRatio = 0.70
dataset = loadCsv(filename)
trainingSet, testSet = splitDataset(dataset, splitRatio)
print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
summaries = summarizeByClass(trainingSet) 
predictions = getPredictions(summaries, testSet)
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: {0}%'.format(accuracy))

Split 768 rows into train=537 and test=231 rows
2
{0.0: [(3.2853025936599423, 3.061392559341601), (109.29394812680115, 26.999679793170106), (68.36311239193084, 18.516064563843425), (20.495677233429394, 14.87352688735698), (70.82132564841498, 101.57150905355682), (30.418155619596536, 8.029309679509803), (0.42277233429394845, 0.28166541774966986), (30.76657060518732, 11.159393282632998)], 1.0: [(4.778947368421052, 3.8837034291070167), (139.45263157894738, 32.372588973507746), (70.9, 20.79772830492558), (23.378947368421052, 17.569622020229204), (102.20526315789473, 136.42671530679505), (35.566315789473684, 7.1736232095545285), (0.5477789473684214, 0.35420887864813866), (36.94736842105263, 10.865087450788568)]}
1 :  [5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0, 0.0] -- 0.0
2 :  [10.0, 115.0, 0.0, 0.0, 0.0, 35.3, 0.134, 29.0, 0.0] -- 1.0
3 :  [8.0, 125.0, 96.0, 0.0, 0.0, 0.0, 0.232, 54.0, 1.0] -- 0.0
4 :  [10.0, 139.0, 80.0, 0.0, 0.0, 27.1, 1.441, 57.0, 0.0] -- 1.0
5 :  [5.0, 166.0, 72.0,